In [ ]:
!nvidia-smi

In [ ]:
import os
import pandas as pd
import ast
import json
import re
import torch
from tqdm import tqdm

In [ ]:
torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

In [ ]:
def process_data(input_str):
    stripped_str = input_str.strip('[]')
    sentences = [s.strip('"') for s in stripped_str.split('","')]
    sentences = ' '.join(sentences)
    return sentences

def get_data(path, system_prompt=None):
    df = pd.read_csv(path)
    df['prompt'] = df['prompt'].apply(process_data)
    df['response_a'] = df['response_a'].apply(process_data)
    df['response_b'] = df['response_b'].apply(process_data)
    return df

In [ ]:
test_path = '/kaggle/input/lmsys-chatbot-arena/test.csv'
test_df = get_data(test_path)
test_df.head()

## Load Gemma-2-9B model
> google/gemma-2-9b-it 

I have already downloaded and stored bfloat16 weights.


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
tokenizer_path = '/kaggle/input/gemma-2-9b-it/gemma-2-9b-it-palash-tokenizer'
model_path = '/kaggle/input/gemma-2-9b-it/gemma-2-9b-it-palash-model'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)

In [ ]:
%%time
model = AutoModelForCausalLM.from_pretrained(model_path, device_map='auto', torch_dtype=torch.bfloat16)

In [ ]:
model

## Let's do a simple inference

In [ ]:
%%time
prompt = 'Write a conversation between gemma and llama llm models'
input_ids = tokenizer(prompt, return_tensors='pt').to('cuda')

outputs = model.generate(**input_ids, max_new_tokens=200)
response = tokenizer.decode(outputs[0])
print(response)

## Let's do inference on test set with some prompt engineering

In [ ]:
test_df

In [ ]:
def get_prompt(query, response_a, response_b):
    prompt = f"""
You are tasked with evaluating two responses generated by different models to determine which one is better. Given a query and two responses (RESPONSE_A from MODEL_A and RESPONSE_B from MODEL_B), you will assess the quality of each response based on relevance, accuracy, completeness, and overall coherence. If both responses are equally good or equally poor, you may declare a tie.

Instructions:

Query: {query}
RESPONSE_A (MODEL_A): {response_a}
RESPONSE_B (MODEL_B): {response_b}

Evaluation Criteria:

Relevance: How well does the response address the query?
Accuracy: Is the information provided correct and reliable?
Completeness: Does the response provide a comprehensive answer?
Coherence: Is the response logically structured and easy to understand?
Output:

If RESPONSE_A is better, output: RESPONSE_A
If RESPONSE_B is better, output: RESPONSE_B
If both responses are equally good or poor, output: TIE

You have to output a single line having either of these words - RESPONSE_A or RESPONSE_B or TIE \n
OUTPUT: 

    """
    return prompt

In [ ]:
def predict(query, response_a, response_b, max_new_tokens=50, do_sample=False, temperature=1.0):
    prompt = get_prompt(query=query, response_a=response_a, response_b=response_b)

    input_ids = tokenizer(prompt, return_tensors='pt').to('cuda')

    outputs = model.generate(**input_ids, max_new_tokens=50, do_sample=do_sample, temperature=temperature)
    response = tokenizer.decode(outputs[0])

    pattern = r"OUTPUT:\s*(RESPONSE_A|RESPONSE_B|TIE)"
    match = re.search(pattern, response)
    if match:
        pred = match.group(1)
    else:
        pred = None
    return pred

In [ ]:
%%time
id_list = []
winner_model_a_list = []
winner_model_b_list = []
winner_tie_list = []
for idx in tqdm(range(0, len(test_df))):
    query_id = test_df.iloc[idx]['id']
    query = test_df.iloc[idx]['prompt']
    response_a = test_df.iloc[idx]['response_a']
    response_b = test_df.iloc[idx]['response_b']
    pred = predict(query, response_a, response_b, max_new_tokens=20, do_sample=True, temperature=0.7)
    id_list.append(query_id)
    if pred is not None:
        if 'A' in pred or 'a' in pred:
            winner_model_a_list.append(1)
            winner_model_b_list.append(0)                    
            winner_tie_list.append(0)
        if 'B' in pred or 'b' in pred:
            winner_model_a_list.append(0)
            winner_model_b_list.append(1)        
            winner_tie_list.append(0)            
    else:
        winner_model_a_list.append(0)
        winner_model_b_list.append(0)        
        winner_tie_list.append(1)
    torch.cuda.empty_cache()

In [ ]:
submission_df = pd.DataFrame({'id': id_list, 'winner_model_a': winner_model_a_list, 'winner_model_b': winner_model_b_list, 'winner_tie': winner_tie_list})

In [ ]:
submission_df

In [ ]:
submission_df.to_csv('submission.csv', index=False)